### **Exemplo de preenchimento matricial**

---

Gabriel Oukawa <br>
Álgebra linear para ciência de dados <br>
2º Semestre de 2025

---


### Conjunto de dados utilizado:

Avaliações de filmes do MovieLens (*https://grouplens.org/datasets/movielens/latest/*)


In [ ]:
# Bibliotecas
import numpy as np
import pandas as pd
from numpy.linalg import svd

In [38]:
# Dados de entrada (id do usuário, id do filme, nota, data)
ratings = pd.read_csv("ratings.csv")

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [52]:
# Construir a matriz de recomendações
R = ratings.pivot(index='userId', columns='movieId', values='rating')

print(f'Tamanho da matriz de entrada: {R.shape}')

np.random.seed(42)

# Número de usuários e filmes para serem preenchidos
n_users  = 100
n_movies = 200

users  = np.random.choice(R.index, n_users, replace=False)
movies = np.random.choice(R.columns, n_movies, replace=False)

R_sub = R.loc[users, movies]

M_true = R_sub.values.astype(float)

Tamanho da matriz de entrada: (610, 9724)


### Aplicação com algoritmo SVD


In [53]:
# Algoritmo de preenchimento, usando SVD clássico
def mask(M, p, seed=42):
    rng = np.random.default_rng(seed)
    obs = ~np.isnan(M)
    keep = obs & (rng.random(M.shape) > p)
    test = obs & (~keep)
    M_obs = M.copy()
    M_obs[test] = np.nan
    fix = ~np.isnan(M_obs)
    return M_obs, fix, test

def svd_fill(M_obs, mask, rank=10, n_iter=50):
    X = M_obs.copy()
    mean_rating = np.nanmean(M_obs)
    X[np.isnan(X)] = mean_rating
    for _ in range(n_iter):
        U, s, Vt = svd(X, full_matrices=False)
        s[rank:] = 0
        X_hat = U @ np.diag(s) @ Vt
        X[~mask] = X_hat[~mask]
        X[mask]  = M_obs[mask]
    return X

# Métrica
def rmse(M_true, M_pred, mask):
    err = M_true[mask] - M_pred[mask]
    return np.sqrt(np.mean(err**2))

# Teste para diferentes frações de dados faltantes
missing_data = [0.1, 0.3, 0.5, 0.7]

for p in missing_data:
    M_obs, S_mask, E_mask = mask(M_true, p)
    M_rec = svd_fill(M_obs, S_mask, rank=10)
    error = rmse(M_true, M_rec, E_mask)
    print(f'Fração de dados faltantes: {int(p*100)}% | RMSE = {error:.3f}')

Fração de dados faltantes: 10% | RMSE = 1.001
Fração de dados faltantes: 30% | RMSE = 1.073
Fração de dados faltantes: 50% | RMSE = 1.053
Fração de dados faltantes: 70% | RMSE = 1.008


### Aplicação com algoritmo SVD relaxado


In [56]:
# Algoritmo de preenchimento, usando SVD relaxado
def mask(M, p, seed=42):
    rng = np.random.default_rng(seed)
    obs = ~np.isnan(M)
    keep = obs & (rng.random(M.shape) > p)
    test = obs & (~keep)
    M_obs = M.copy()
    M_obs[test] = np.nan
    fix = ~np.isnan(M_obs)
    return M_obs, fix, test

def svd_relaxed(M, fix, rank=10, omega=0.5, n_iter=50):
    X = M.copy()
    for _ in range(n_iter):
        U, s, Vt = svd(X, full_matrices=False)
        s[rank:] = 0.0
        X_hat = U @ np.diag(s) @ Vt
        X[~fix] = (1 - omega) * X[~fix] + omega * X_hat[~fix]
    return X


# Métrica
def rmse(M, M_hat, test):
    return np.sqrt(np.mean((M[test] - M_hat[test])**2))

# Teste para diferentes frações de dados faltantes
missing = [0.1, 0.3, 0.5, 0.7]

for p in missing:
    M_obs, fix, test = mask(M_true, p)
    mu = np.nanmean(M_obs)
    M0 = np.where(fix, M_obs, mu)
    M_rec = svd_relaxed(M0, fix, rank=10, omega=0.6, n_iter=60)
    err = rmse(M_true, M_rec, test)
    print(f'Fração de dados faltantes: {int(p*100)}% | RMSE = {err:.3f}')

Fração de dados faltantes: 10% | RMSE = 0.997
Fração de dados faltantes: 30% | RMSE = 1.073
Fração de dados faltantes: 50% | RMSE = 1.052
Fração de dados faltantes: 70% | RMSE = 1.008
